# Automatic Issues Triaging with Llama

We utilize an off-the-shelf Llama model to analyze, generate insights, and create a report for better understanding of the state of a repository. 

This notebook walks you through the tool's working. 

## Setup

In [ ]:
!git clone https://github.com/meta-llama/llama-recipes

%cd recipes/use_cases/github_triage

!pip install -r requirements.txt

In [6]:
import yaml
import os

from llm import run_llm_inference
from utils import fetch_repo_issues, validate_df_values
from plots import draw_all_plots
from pdf_report import create_report_pdf
from triage import generate_executive_reports, generate_issue_annotations

### Set access keys and tokens

Set your GitHub token for API calls. Some privileged information may not be available if you don't have push-access to the target repository.

Set your groq token for inference. Get one at https://console.groq.com/keys

In [2]:
github_token = input("Enter your Github API token")
groq_token = input("Enter your Groq token")

with open("config.yaml", "r") as f:
  CFG = yaml.safe_load(f)
CFG['github_token'] = github_token
CFG['model']['groq']['key'] = groq_token
with open("config.yaml", "w") as f:
  yaml.dump(CFG, f)

### Set target repo and period to analyze

In [12]:
repo_name = input("What repository do you want to analyze? (eg: meta-llama/llama-recipes)")
start_date = input("Start analysis (eg: 2024-08-23): ")
end_date = input("End analysis (eg: 2024-08-30): ")

out_folder = f'output/{repo_name}/{start_date}_{end_date}'
os.makedirs(out_folder, exist_ok=True)

print(f"Running AutoTriage for {repo_name} from {start_date} to {end_date}")

Running AutoTriage for pytorch/pytorch from 2024-08-28 to 2024-08-28


### Fetch issues from the repository

Use the github API to retrieve issues (including the full discussion on them) and store it in a dataframe.

In [13]:
issues_df = fetch_repo_issues(repo_name, start_date, end_date)

print(f"[Showing 5 out of {issues_df.shape[0]} rows]\n")
print(issues_df.head())

Fetching issues on pytorch/pytorch from 2024-08-28 to 2024-08-28
Fetched 26 issues on pytorch/pytorch from 2024-08-28 to 2024-08-28


[Showing 5 out of 26 rows]

         repo_name  number                                          html_url  \
0  pytorch/pytorch  134637  https://github.com/pytorch/pytorch/issues/134637   
1  pytorch/pytorch  134640  https://github.com/pytorch/pytorch/issues/134640   
2  pytorch/pytorch  134641  https://github.com/pytorch/pytorch/issues/134641   
3  pytorch/pytorch  134642  https://github.com/pytorch/pytorch/issues/134642   
4  pytorch/pytorch  134644  https://github.com/pytorch/pytorch/issues/134644   

   closed  num_comments            created_at  \
0   False             0  2024-08-28T01:14:14Z   
1   False             0  2024-08-28T01:58:33Z   
2   False             1  2024-08-28T02:00:59Z   
3   False             4  2024-08-28T02:15:07Z   
4   False             0  2024-08-28T02:25:25Z   

                                          discussion  
0  LSTM inference c++ threads block on DropoutSta...  
1  Meet “No module named'tools. setup_helpers‘“ w...  
2  nn.Module.to(memory_format= 

### Use Llama to generate the annotations for this data

We use the `parse_issues` prompt defined in `config.yaml` to generate annotations and other metadata for each issue thread.

`assign_category` tags each issue with the most relevant category defined in the prompt.

In [29]:
print(f"Prompt for generating annotations:\n{CFG['prompts']['parse_issue']}\n")
print(f"Prompt for categorizing issues:\n{CFG['prompts']['assign_category']}\n")
print(f"Issues can be categorized into: {eval(CFG['prompts']['assign_category']['json_schema'])['properties']['report']['items']['properties']['theme']['enum']}")

Prompt for generating annotations:
{'system': 'You are an expert maintainer of an open source project. Given some discussion threads, you must respond with a report in JSON. Your response should only contain English, and you may translate if you can.', 'json_schema': '{ "type": "object", "properties": { "summary": { "description": "Summary of the issue and discussion along with any details about platform or tooling.", "type": "string" }, "possible_causes": { "type": "array", "items": { "type": "string" } }, "remediations": { "description": "How we can improve the code or documentation to prevent this issue.", "type": "array", "maxItems": 2, "items": { "type": "string" } }, "component": { "description": "The specific module or component affected by the issue", "type": "string" }, "sentiment": { "type": "string", "enum": ["positive", "negative", "neutral"] }, "issue_type": { "description": "Any issue not related to LLMs, Llama or code in this repository should be marked as \\"invalid\\""

In [22]:
# Generate annotations and metadata
annotated_issues, theme_counts = generate_issue_annotations(issues_df)
# Validate and save generated data
annotated_issues = validate_df_values(annotated_issues, out_folder, 'annotated_issues')

Data saved to output/pytorch/pytorch/2024-08-28_2024-08-28/annotated_issues.csv


The generated annotations include a summary and category (theme, component affected) for each issue, that can help triagers quickly identify the problem and the right POC to take a look.

In [30]:
print(f"metadata generated by LLM: {set(annotated_issues.columns).difference(set(issues_df.columns))}\n\n")
print(f"[Showing 5 out of {annotated_issues.shape[0]} rows]\n")
print(annotated_issues.head())

metadata generated by LLM: {'sentiment', 'themes', 'summary', 'remediations', 'component', 'possible_causes', 'severity', 'op_expertise', 'issue_type'}


[Showing 5 out of 26 rows]

         repo_name  number                                          html_url  \
0  pytorch/pytorch  134637  https://github.com/pytorch/pytorch/issues/134637   
1  pytorch/pytorch  134640  https://github.com/pytorch/pytorch/issues/134640   
2  pytorch/pytorch  134641  https://github.com/pytorch/pytorch/issues/134641   
3  pytorch/pytorch  134642  https://github.com/pytorch/pytorch/issues/134642   
4  pytorch/pytorch  134644  https://github.com/pytorch/pytorch/issues/134644   

   closed  num_comments            created_at  \
0   False             0  2024-08-28T01:14:14Z   
1   False             0  2024-08-28T01:58:33Z   
2   False             1  2024-08-28T02:00:59Z   
3   False             4  2024-08-28T02:15:07Z   
4   False             0  2024-08-28T02:25:25Z   

                                          

### Use Llama to generate high-level insights

The above data is good for OSS maintainers and developers to quickly address any issues. The next section will synthesize this data into high-level insights that can enable decisioning about management of this repository.

In [31]:
print(f"Prompt for generating high-level overview:\n\n{CFG['prompts']['get_overview']}")

Prompt for generating high-level overview:

{'system': 'You are not only an experienced Open Source maintainer, but also an expert at paraphrasing raw data into clear succinct reports. Draft a concise report about the issues in this open source repository. Include an executive summary that provides an overview of the challenges faced, any open questions or decisions to be made, or actions that we can take. Group issues together if they ladder up to the same overall challenge, summarize the challenges and include any actionable resolutions we can take (more information in the \\"remediations\\" sections). Use your experience and judgement to ignore issues that are clearly unrelated to the open source project. Ensure the output is in JSON.', 'json_schema': '{ "type": "object", "properties": { "executive_summary": { "description": "An executive summary of the analysis", "type": "string" }, "open_questions": { "description": "Any open questions or decisions that the product team needs to m

In [19]:
# Generate high-level analysis
challenges, overview = generate_executive_reports(annotated_issues, theme_counts, repo_name, start_date, end_date)
# Validate and save generated data
challenges = validate_df_values(challenges, out_folder, 'challenges')
overview = validate_df_values(overview, out_folder, 'overview')

Generating high-level summaries from annotations...
Identifying key-challenges faced by users...
Data saved to output/pytorch/pytorch/2024-08-28_2024-08-28/challenges.csv
Data saved to output/pytorch/pytorch/2024-08-28_2024-08-28/overview.csv


We identify key areas that users are challenged by and tag which issues represent those challenges.

In [20]:
print(f"[Showing 5 out of {challenges.shape[0]} rows]\n")
print(challenges.head())

[Showing 5 out of 5 rows]

         repo_name  start_date    end_date                    key_challenge  \
0  pytorch/pytorch  2024-08-28  2024-08-28          Performance Regressions   
1  pytorch/pytorch  2024-08-28  2024-08-28             Compatibility Issues   
2  pytorch/pytorch  2024-08-28  2024-08-28         Security Vulnerabilities   
3  pytorch/pytorch  2024-08-28  2024-08-28  Tensor Parallelism and Autograd   
4  pytorch/pytorch  2024-08-28  2024-08-28                     CUDA Support   

            affected_issues  \
0          [134679, 134686]   
1  [134640, 134682, 134684]   
2                  [134664]   
3          [134668, 134676]   
4          [134682, 134684]   

                                     possible_causes  \
0  [Changes in the torch or torchvision libraries...   
1  [Incompatible version of caffe2 with the curre...   
2  [Using the affected version of protobuf (3.20....   
3  [Lack of understanding of tensor parallelism i...   
4  [cuDNN version incompatibili

As the name suggests, the `overview` dataframe contains columns that provide information about the overall activity in the repository during this period, including:
* an executive summary of all the issues seen during this period
* how many issues were created, discussed and closed
* what are some open questions that the maintainers should address
* how many issues were seen for each theme etc.

In [32]:
print(f"[Showing 5 out of {overview.shape[0]} rows]\n")
print(overview.head())

[Showing 5 out of 1 rows]

         repo_name  start_date    end_date  issues_created  open_discussion  \
0  pytorch/pytorch  2024-08-28  2024-08-28              26                8   

   closed_discussion  open_no_discussion  closed_no_discussion  \
0                  0                  18                     0   

                                      open_questions  \
0  [What is the root cause of the performance reg...   

                                   executive_summary  ...  \
0  The PyTorch repository is facing various chall...  ...   

   themes_count_model_loading  themes_count_model_fine_tuning_and_training  \
0                           3                                            3   

   themes_count_model_inference  \
0                             3   

   themes_count_distributed_training_and_multi_gpu  \
0                                                3   

   themes_count_performance_and_optimization  \
0                                          5   

   themes_c

### Visualizing the data

Based on this data we can easily create some plots to graphically understand the activity in the repo.

Some additional data can be accessed via the github API, but this requires you to have push-access to this repo.

In [33]:
# Create graphs and charts
plot_folder = out_folder + "/plots"
os.makedirs(plot_folder, exist_ok=True)
draw_all_plots(repo_name, plot_folder, overview)

Plotting traffic trends...


Github fetch failed for <function plot_views_clones at 0x13837a3e0>. Make sure you have push-access to pytorch/pytorch!
Github fetch failed for <function plot_high_traffic_resources at 0x13ed59580>. Make sure you have push-access to pytorch/pytorch!
Github fetch failed for <function plot_high_traffic_referrers at 0x13ed84860>. Make sure you have push-access to pytorch/pytorch!


Plotting issue trends...


### Putting it together

Now that we have all the data and insights, we can create a PDF report

In [ ]:
    exec_summary = overview['executive_summary'].iloc[0]
    open_qs = overview['open_questions'].iloc[0]
    key_challenges_data = challenges[['key_challenge', 'possible_causes', 'remediations', 'affected_issues']].to_dict('records')
    create_report_pdf(repo_name, start_date, end_date, key_challenges_data, exec_summary, open_qs, out_folder)